In [1]:
%load_ext autoreload

In [2]:
%autoreload 1

In [3]:
%aimport src.config

In [4]:
import numpy as np
from pathlib import Path
from time import time
import json
import pandas as pd
from xml.etree.ElementTree import iterparse

In [5]:
from src.config import data_dir

#### Process raw SMS data

In [51]:
filename = "karim-sms-allow.xml"
source = data_dir / filename
data = []
for event, elem in iterparse(source):
    if elem.tag == "sms":
        #if any(elem.attrib["body"]==r["text"] for r in data):
        #    continue
        record = {}
        record["text"] = elem.attrib["body"]
        record["contact_name"] = elem.attrib["contact_name"]
        record["address"] = elem.attrib["address"]
        record["timestamp"] = int(elem.attrib["date"])
        record["type"] = elem.attrib["type"]
        data.append(record)

In [55]:
df = pd.DataFrame(data)
df.to_excel(data_dir / "karim-sms-allow.xlsx", index=False)

#### Read labeled data

In [6]:
labeled_filename = "karim-sms-allow-labeled.xlsx"
labeled = pd.read_excel(data_dir / labeled_filename, sheet_name="total sms")

In [20]:
# Check dimensionality and class imbalance
labeled.shape
labeled.label.value_counts(normalize=True).round(5)*100
labeled.tail()

(3228, 7)

0    82.187
1    17.813
Name: label, dtype: float64

,address,contact_name,text,timestamp,type,label,service
3223,+380631576335,Dad,I can't talk right now. I'll call you back sho...,1524840349700,2,0,1
3224,+380937808130,Танька,Я вже на зв’язку з 28.04 02:40. Передзвоніть ...,1524872412545,1,0,1
3225,lifecell,(Unknown),Я намагався Вам додзвонитися. Передзвоніть мен...,1524909347947,1,0,1
3226,+380638197928,Mom,"У Вас пропущених дзвінків: 1 від мене, останн...",1524915699942,1,0,1
3227,+380442251208,(Unknown),"У Вас пропущених дзвінків: 1 від мене, останн...",1525069655264,1,0,1
